# Mini-Project: Random Walk for a Price / Demand Index

This notebook is another optional **mini-project** for the course.

It is designed to grow together with the 4 sessions:
we will build a simple stochastic model — a **random walk** — that can represent:

- a price index (e.g. spot price of grain or fuel),
- a demand index,
- a congestion / capacity tightness index,
- or any quantity that moves up and down over time.

We will build the simulator in four stages:

1. **Session 1:** Understand the model and choose data structures.
2. **Session 2:** Implement a simple deterministic walk with a loop.
3. **Session 3:** Add randomness and wrap logic into functions.
4. **Session 4:** Save results to a CSV file and make basic plots.


---
## 1. Session 1 – Understanding the Idea (No Code Yet)

### Informal story

We track an index \(X_t\) over time (for example, a price index):

- At time 0, the index has some initial value \(X_0\) (e.g. 100).
- Each day, the index moves **up or down** by a small amount.
- The movement is partly random: we cannot predict the exact path,
  only the **distribution of possible futures**.

In discrete time, a very simple model is:

\[
X_{t+1} = X_t + \epsilon_t,
\]

where the "step" \(\epsilon_t\) is a random small number, for example \(-1, 0, +1\).

### Questions to think about

1. How would you store the value of the index at each time step (for a single path)?  
2. How would you store **several** different paths (several simulated futures)?  
3. What summary statistics might be interesting?
   - minimum and maximum value,
   - value at the final time,
   - number of times the index crosses a threshold (e.g. goes above 120).

You can discuss this with a partner or write your ideas in a text cell.


> **Optional thought experiment:**  
> Imagine that the index represents a fuel price index, starting at 100.  
> If it randomly goes up or down by 1 each day, what kind of futures do you expect?
> Constantly drifting away? Staying around 100? Hitting very high or very low values?


---
## 2. Session 2 – Core Loop for a Simple Walk (Deterministic Steps)

We start with a very simple version:

- Fixed initial value \(X_0\).
- A **deterministic** step each day (for now), not random.
- We just want to build and understand the loop structure that updates \(X_t\).

Later, we will replace the fixed steps by random steps.


In [ ]:
# Parameters

T = 20      # number of days (time steps)
x0 = 100    # initial index value

x = x0
path = [x]

print("Day | Value")
print("-----------")
print(f"{0:3d} | {x}")

for day in range(1, T + 1):
    # TODO: choose how the index moves.
    # For now, try a simple fixed pattern, e.g. always +1,
    # or alternate +1, -1.
    step = 1   # <-- replace this
    
    x = x + step

    # Optional: enforce a floor at 0
    if x < 0:
        x = 0

    path.append(x)
    print(f"{day:3d} | {x}")

> **Tasks:**
>
> 1. Start with `step = 1` and run the cell. What happens to the path?
> 2. Make `step` alternate between +1 and -1 (for example, using `if day % 2 == 0`).
> 3. Make `step` depend on whether `x` is above or below a threshold (e.g. 100).
>    This is a toy version of **mean reversion**.
>
> At this stage we are still completely deterministic — no randomness yet.


---
## 3. Session 3 – Random Steps and Functions

Now we will:

1. Introduce **randomness** in the steps using the `random` module.
2. Wrap the logic in a function `simulate_path`.
3. Simulate **many paths** and compute some simple statistics.


In [ ]:
import random

def random_step():
    """Return a random step for the random walk.

    For now, we choose uniformly from {-1, 0, +1}.
    You can change this later if you like.
    """
    return random.choice([-1, 0, 1])

In [ ]:
def simulate_path(T=30, x0=100):
    """Simulate one random walk path.

    Parameters
    ----------
    T : int
        Number of time steps.
    x0 : float or int
        Initial value of the index.

    Returns
    -------
    path : list of numbers
        The values of the index at each time step, length T+1.
    """
    x = x0
    path = [x]

    for t in range(T):
        step = random_step()
        x = x + step

        # Floor at 0 (optional, to avoid negative prices)
        if x < 0:
            x = 0

        path.append(x)

    return path

In [ ]:
# Try simulating a single path
path = simulate_path(T=30, x0=100)
path[:10], len(path)

> **Task:**  
> Run `simulate_path` several times and observe how the trajectory changes.
>
> - What is the minimum and maximum value in each simulation?
> - What is the final value after T steps?


In [ ]:
def simulate_many_paths(N=100, T=30, x0=100):
    """Simulate N independent random walk paths."""
    paths = []
    for i in range(N):
        path = simulate_path(T=T, x0=x0)
        paths.append(path)
    return paths

# Example usage
paths = simulate_many_paths(N=100, T=30, x0=100)

final_values = [p[-1] for p in paths]
print("Number of paths:", len(paths))
print("Average final value:", sum(final_values) / len(final_values))
print("Minimum final value:", min(final_values))
print("Maximum final value:", max(final_values))

> **Tasks:**
>
> 1. Change `N` and `T` and see how the distribution of final values changes.
> 2. Compute the proportion of paths whose final value is:
>    - above 110,
>    - below 90.
> 3. Interpret these as **probabilities** of different scenarios.


### Optional: Biased Random Walk

We can introduce a small **drift** by making upward steps more likely than downward steps.

For example, with probability `p` we step +1, otherwise -1.


In [ ]:
def random_step_biased(p=0.55):
    """Return +1 with probability p, else -1."""
    if random.random() < p:
        return 1
    else:
        return -1

def simulate_path_biased(T=30, x0=100, p=0.55):
    x = x0
    path = [x]
    for t in range(T):
        step = random_step_biased(p=p)
        x = x + step
        if x < 0:
            x = 0
        path.append(x)
    return path

# Quick test
biased_path = simulate_path_biased(T=30, x0=100, p=0.55)
biased_path[:10]

> **Optional tasks:**
>
> 1. Compare the average final values of unbiased vs biased walks.
> 2. For the biased walk, how does the probability of ending above 120 change?


---
## 4. Session 4 – Saving Results and Plotting

Now we will:

1. Plot one or several paths using `matplotlib`.
2. Save one path to a CSV file.


In [ ]:
import matplotlib.pyplot as plt

# Plot a single unbiased path
path = simulate_path(T=50, x0=100)

plt.plot(path)
plt.xlabel("Day")
plt.ylabel("Index value")
plt.title("Random walk path")
plt.show()

In [ ]:
# Plot several paths on the same figure

paths = simulate_many_paths(N=5, T=50, x0=100)

for p in paths:
    plt.plot(p)

plt.xlabel("Day")
plt.ylabel("Index value")
plt.title("Several random walk paths")
plt.show()

> **Tasks:**
>
> 1. Change `N` and `T` and re-run the plots.
> 2. Plot both unbiased and biased paths on the same figure (using different colours or line styles).
> 3. Visually compare how often the paths cross a high threshold (e.g. 120).


In [ ]:
# Save a single path to CSV

path = simulate_path(T=50, x0=100)
filename = "random_walk_path.csv"

with open(filename, "w") as f:
    f.write("day,value\n")
    for day, value in enumerate(path):
        f.write(f"{day},{value}\n")

print(f"Wrote path to {filename}")

> In Google Colab, you can download the file using:
>
> ```python
> from google.colab import files
> files.download("random_walk_path.csv")
> ```
>
> In a local Jupyter environment, you can inspect the file in your working directory.


---
## 5. Extensions (Optional, For Curious Minds)

Here are some ideas if you want to push the model further:

1. **Barrier hitting time:**  
   Simulate many paths and estimate the probability that the index hits 0
   (or a high level like 130) at least once before time T.

2. **Geometric random walk (prices):**  
   Instead of adding a step, multiply by a random factor, e.g.
   \(P_{t+1} = P_t (1 + \epsilon_t)\) where \(\epsilon_t\) is a small random percentage.

3. **Scenario comparison:**  
   Interpret the index as a cost or price index. Compare two different volatility levels
   and estimate the chance that costs exceed a given threshold.

4. **Export-and-analyse later:**  
   Save results for many paths (one file per path or a wide CSV), and analyse them in the
   follow-up course using pandas.
